# Predicting Airbnb Price in European Cities 

---

**Group 15** <br>
Mathew <br>
Marcus <br>
Zahra <br>
Alizah <br>

---

# Introduction

Airbnb has revolutionized the short-term rental market, making tourism and travel more accessible and convenient. As a popular alternative to traditional hotels, hosts can monetize their properties by listing it on the platform, enabling guests to customize and enrich their travel experiences. In Europe, tourism plays a significant role in many cities’ economies, and understanding the factors that influence listing prices will help explain what truly adds value to a stay. 

#### Literature Review

Listing reputation was associated with a negative effect on price, as hosts with lower prices may attract more guests and reviews ([Toader, Victor, et al, 2021](https://doi.org/10.1080/1331677x.2021.1962380)). The same study also associated the number of bedrooms and guest capacity with the largest price changes. 

Listing prices have also been concluded to be spatially dependent, changing with distance from city centre, distance to the nearest metro station, and distances to major attractions ([Gyódi, Karol, and Łukasz Nawaro, 2021](https://doi.org/10.1016/j.tourman.2021.104319)). The authors determined that Airbnb prices decrease further from city centre and metro stations, but increase in the vicinity of popular tourist attractions and restaurants.

#### **Our Question:** Which factors are most useful for predicting the price of an Airbnb listing?

Confirming the importance of location and other factors, and being able to estimate listing prices with good accuracy, is helpful for both hosts and guests. Beyond tourism management, understanding Airbnb prices will also be informative for urban planning, expanding upon previous studies. 

## Data Description:
Our analysis will use the dataset [Airbnb Prices in European Cities](https://www.kaggle.com/datasets/thedevastator/airbnb-prices-in-european-cities/data).

The data appears to be aggregated from Airbnb listings from several major European cities. The cities we will focus on are London, Rome, and Budapest, on weekends and weekdays, for a total of `23042` observations. 

There are 20 variables in the original dataset, but we drop `id`, `attr_index`, `attr_index_norm`, `rest_index`, and `rest_index_norm` as they aren’t explained in detail. This leaves us with 15 variables:

| Column name   | Description   | Data Type     |
| ------------- | ------------- | ------------- |
| `realSum` | The total price of the Airbnb listing (Euros). | Numeric |
| `room_type` | The type of room being offered (e.g. private, shared, etc.). | Categorical |
| `room_shared` | Whether the room is shared or not. | Boolean |
| `room_private` | Whether the room is private or not. | Boolean |
| `person_capacity` | The maximum number of people that can stay in the room. | Numeric |
| `host_is_superhost` | Whether the host is a superhost or not. | Boolean |
| `multi` | Whether the listing is for multiple rooms or not. | Boolean |
| `biz` | Whether the listing is for business purposes or not. | Boolean |
| `cleanliness_rating` | The cleanliness rating of the listing. | Numeric |
| `guest_satisfaction_overall` | The overall guest satisfaction rating of the listing. | Numeric |
| `bedrooms` | The number of bedrooms in the listing. | Numeric |
| `dist` | The distance from the city centre. | Numeric |
| `metro_dist` | The distance from the nearest metro station. | Numeric |
| `lng` | The longitude of the listing. | Numeric |
| `lat` | The latitude of the listing. | Numeric |


The following two variables will be added to the data when imported: 

| Column name   | Description   | Data Type     |
| ------------- | ------------- | ------------- |
| `city` | The city location of the Airbnb listing. | Categorical |
| `isWeekend` | Whether the listing is for weekends or not. | Boolean |


# Methods and Results
## a) Exploratory Data Analysis
We start by reading our data and drop the columns we do not need (`id`, `attr_index`, `attr_index_norm`, `rest_index` and `rest_index_norm`). We will also drop `lng` and `lat`, because they must be considered in a combined and discrete manner to be meaningful. We will also correct the following columns:

- `room_type` from \<chr\> to factor (\<fct\>) 
- `multi` & `biz` from \<dbl\> to logicals (\<lgl\>) 

In [ ]:
install.packages("mltools")
install.packages("glmnet")
install.packages("leaps")
# You may need to run these functions if certain packages are not already installed.

In [ ]:
# Libraries & Packages
library(tidyverse)
library(dplyr)
library(mltools)
library(rsample)
library(MASS)
library(car)

In [ ]:
# Main developer: Alizah
# Contributor: Marcus (modified read function)

# Function to format & read data: 
# select vars from dataset and add city name + if it's weekend data
tidy_data <- function(path, cityName, isWeekend) {
    data <- read_csv(path, show_col_types = FALSE) %>%
            dplyr::select(-...1, -attr_index, -attr_index_norm, -rest_index, 
                          -rest_index_norm, -lng, -lat) # Drop columns
    data$city <- as.factor(cityName)        # The city location of the Airbnb listing (Categorical)
    data$isWeekend <- as.logical(isWeekend) # Whether the listing is for weekends or not. (Boolean)
    return(data)
}

# Import and format data with tidy_data()
london_weekdays <- tidy_data("data/london_weekdays.csv", "London", 0)
london_weekends <- tidy_data("data/london_weekends.csv", "London", 1)
rome_weekdays <- tidy_data("data/rome_weekdays.csv", "Rome", 0)
rome_weekends <- tidy_data("data/rome_weekends.csv", "Rome", 1)
budapest_weekdays <- tidy_data("data/budapest_weekdays.csv", "Budapest", 0)
budapest_weekends <- tidy_data("data/budapest_weekends.csv", "Budapest", 1)

# Merge all the data together
data <- rbind(london_weekdays, london_weekends, rome_weekdays, rome_weekends, budapest_weekdays, budapest_weekends)

# View the head and tail of the data
head(data, 3)
tail(data, 3)

In [ ]:
# Main developer: Matthew
# Contributor: Zahra (Style Guide)

# Correct the columns with wrong type
data <- data %>%
    mutate(
        #Convert to factor
        across(c(room_type), as.factor),
        
        #Convert to logical
        across(c(multi, biz), as.logical)
    )

# View the corrected head of the data
head(data, 3)

In [ ]:
# Main developer: Marcus
# Contributor: Alizah (organized output)

# Inspect data structure
str(data)
cat("\n")

# Check for missing/NA values
cat("NA values present?: ", anyNA(data))
cat("\n")

# Compute summary statistics & check numbers
cat("Data dimensions: ", dim(data))
cat("\n\n")
cat("Summary of realSum")
summary(data$realSum)

Above, we see `realSum` with a maximum value of `15499.89`, minimum of `34.78`, and a mean of `268.47`. This suggests the presence of outliers. Let’s visualize the distribution of `realSum` with a histogram to further examine outliers.

In [ ]:
# Main developer: Marcus
# Contributor: Zahra (Style Guide)

# Adjust size of plots
options(repr.plot.width = 8, repr.plot.height = 5)

# Plot a histogram of our response variable realSum
realSum_hist <- data %>%
    ggplot(aes(x = realSum)) +
    geom_histogram(fill = "gray", color = "black") +
    labs(
        title = "Histogram of Airbnb Prices", 
        x = "Total Airbnb Price (Euros)", 
        y = "Count"
    ) +
    theme(
        axis.text = element_text(size = 16), 
        axis.title = element_text(size = 16),
        title = element_text(size = 20)
    )

realSum_hist

Above, we see high outliers in `realSum`. Let’s filter them by removing the 95th percentile, grouped by cities, and plot again.

In [ ]:
# Main developer: Matthew
# Contributor: Zahra (Style Guide)

# Filtering Outliers from each city by removing listings above the 95th percentile
data_filt <- data %>% 
    group_by(city) %>%
    filter(realSum <= quantile(realSum, 0.95, na.rm = TRUE)) %>%
    ungroup()

In [ ]:
# Main developer: Marcus
# Contributor: Zahra (Style Guide and Aesthetics)

# Plot filtered histogram with Mean and 95th Percentile lines
realSum_hist_filt <- data_filt %>%
    ggplot(aes(x = realSum)) +
    geom_histogram(fill = "gray", color = "black") +
    geom_vline(xintercept = mean(data$realSum), color = "red", linetype = "dashed", size = 1) +
    geom_vline(xintercept = quantile(data$realSum, 0.95), color = "blue", linetype = "dashed", size = 1) +
    scale_x_continuous(breaks = seq(0, 1000, 100)) +
    ggtitle("Histogram of Airbnb Prices at 95th percentile\nwith 95% threshold (blue) and mean price (red)") +
    labs(x = "Total Airbnb Price", y = "Count") +
    theme(
        axis.text = element_text(size = 15), 
        axis.title = element_text(size = 15),
        title = element_text(size = 16)
    )
    
realSum_hist_filt

Our filtered data appears right-skewed. 95% of Airbnb price listings fall below about `900` grouped by city, but fall below about `640` for the entire dataset, so we need to explore `city`. There’s also low outliers under `100`, so we’ll drop them by filtering listings below the 5th percentile.

In [ ]:
# Main developer: Matthew

# Filter again to exclude lower 5%
data_filt <- data_filt %>%
    group_by(city) %>%
    filter(realSum >= quantile(realSum, 0.05, na.rm = TRUE)) %>%
    ungroup()

In [ ]:
# Main developer: Matthew
# Contributor: Zahra (Style Guide)

#Set plot size
options(repr.plot.height = 6, repr.plot.width = 8)

# Creating a Boxplot to examine distribution of listing prices by city
city_listing_plot <-
    data_filt %>%
    ggplot(aes(x = city, y = realSum, fill = city)) +
    geom_boxplot()+
    stat_summary(
        aes(x = city, y = realSum),
        fun = mean, 
        geom = "point", 
        colour = "yellow", 
        shape = 18, 
        size = 5
    ) +
    ggtitle("Distribution of Listing Prices in Different Cities") +
    xlab("Cities") +
    ylab("Listing Prices (EUR)") +
    theme(
        text = element_text(size = 12),
        plot.title = element_text(face = "bold"),
        axis.title = element_text(face = "bold")
    )
city_listing_plot

The box plot above shows the distribution of listing prices across London, Rome and Budapest. 

- Budapest (Blue) appears to have the smallest variability as well as the lowest median and mean.
- London (Red) has the largest variability and the highest median and mean.
- Lastly, Rome (Green) has a variability, median and mean higher than Budapest but lower than London.
- Budapest and Rome share a relatively similar spread, median and mean of listing prices to one another. However, London's wider distribution and higher median and mean may affect the goodness of fit of the model.

Lastly, let's plot a correlation matrix to explore all associations between inputs.

In [ ]:
# Main Developer: Marcus (Referenced from STAT 301 Tutorial 3)

# Select only numeric & drop response, then get melted correlation matrix
corr_matrix <- 
    data_filt %>%
    dplyr::select(where(is.numeric)) %>%
    dplyr::select(-realSum) %>%
    cor() %>%
    as_tibble(rownames = 'var1') %>%
    pivot_longer(-var1, names_to = "var2", values_to = "corr")

head(corr_matrix, 3)

In [ ]:
# Main Developer: Marcus (Referenced from STAT 301 Tutorial 3)

options(repr.plot.width = 12, repr.plot.height = 8) 
# You may need this to generate the plot
options(jupyter.plot_mimetypes = "image/png")

plot_corr_matrix <- corr_matrix %>%
    ggplot(aes(var1, var2)) +
    geom_tile(aes(fill = corr), color = "white") +
    scale_fill_distiller("Correlation Coefficient \n",
        palette =  "YlOrRd",
        direction = 1, 
        limits = c(-1.0, 1.0)
    ) +
    labs(x = "Input Variable 1", y = "Input Variable 2") +
    theme_minimal() +
    theme(
        axis.text.x = element_text(angle = 45, vjust = 1, size = 14, hjust = 1),
        axis.text.y = element_text(vjust = 1, size = 14, hjust = 1),
        legend.title = element_text(size = 18),
        legend.text = element_text(size = 12),
        legend.key.size = unit(1.5, "cm")
    ) +
    coord_fixed() +
    geom_text(aes(var1, var2, label = round(corr, 2)), color = "black", size = 6)

plot_corr_matrix

Above, there seem to be moderate (< 0.75) correlations between `person_capacity` and `bedrooms`, `metro_dist` and `dist`, and `guest_satisfaction_overall` and `cleanliness_rating`.

## b) Analysis 

Our question involves both inference and prediction. We separated the data into testing and training sets to prevent post-inference problems and to get an unbiased understanding of our model's scalability and performance.

### Splitting & Initial Fit

We used `initial_split` (from `rsample`) to randomly split our dataset, stratified by `realSum` with 80/20 training-test split. For the rest of the analysis, we ensured the testing data was not leaked into the model.

First, we fit a baseline (intercept-only) model as a point of comparison. Our main model used `stepAIC` to systematically select the most important factors for predicting our listing price. 

We chose `stepAIC` with forward selection, because `stepAIC` takes multicollinearity into account unlike LASSO regularization, which may randomly drop predictors with similar model contributions. It also considers categorical variables as a whole rather than by each dummy variable created, in contrast to `regsubsets`.

`stepAIC` with forward selection adds predictors one by one to the baseline model, choosing the predictor that yields the largest drop in AIC (a metric used to compare model fit). This process repeats until no further AIC improvement is possible, balancing overfitting and underfitting to find the least complex model with the best fit. 


### Checking Assumptions:
We assumed independence of observations. For additive models, we assumed no combined interactions (between predictors) that influence the response.

We check our model assumptions for linearity, normality, homoscedasticity and collinearity. Although we have enough observations (23042) to apply the CLT and assume normality, it is good to check.

Residuals vs. Fitted Plot (Figure 1):
- To check linearity, the plot should have the same number of points above/below the reference line at any value of fitted values. 
- To check homoscedasticity, the plot should not look like a “cone”.

Q-Q Plot (Figure 2): 
- To check normality, the plot should show points that align with the reference line.

To check for multicollinearity, we used `vif` to find highly collinear variables.

### Testing & Evaluation:
To validate assumptions, we log-transformed `realSum` and dropped collinear variables `room_shared` and `room_private`. We also standardized the numerical predictors.

Then, we used the transformed data with `stepAIC` and validated assumptions (Figure 3 & 4). We extracted selected predictors and fit them on an `lm` model.

We called `predict` with our finalized model and testing dataset and calculated the Root-Mean Squared Error using `rmse` (from `mltools`). RMSE is more sensitive to large errors because it squares them, helping us measure price prediction accuracy to a finer extent (every euro counts). Lower RMSE suggests better model accuracy. We also plotted our predicted and actual values as a visual aid of our model's performance (Figure 5). 

## c) Results

### Splitting + Initial Fit

In [ ]:
# Main developer: Alizah

set.seed(1515)

# Split data between Training and Testing data, not standardized
data_split <- initial_split(data_filt, prop = 0.8, strata = realSum)
training_data <- training(data_split)
testing_data <- testing(data_split)

In [ ]:
# Main developer: Alizah

# Use the Intercept-Only Model as a Baseline for performance
baseline_model <- lm(realSum ~ 1, data = training_data)

# Run forward selection starting from baseline model (full_model is the upper boundary)
full_model <- lm(realSum ~ ., data = training_data)

step_model <- stepAIC(
    baseline_model, 
    scope = list(lower = baseline_model, upper = full_model),
    direction = "forward",
    trace = FALSE) # minimizes output

# THIS CELL MAY TAKE A WHILE TO RUN
# RESULTS SHOWN IN NEXT CELL

In [ ]:
# Main developer: Alizah

# Viewing the selected predictors
final_formula = formula(step_model)
final_formula

### Checking Assumptions

In [ ]:
# Main developer: Alizah
# Collaborator: Zahra (Histogram of Residuals)

# Get Residuals
residuals <- resid(step_model)
fitted_vals <- fitted(step_model)

# 1. Residuals vs Fitted plot (for linearity & homoscedasticity)
plot(fitted_vals, residuals, xlab = "Fitted values", ylab = "Residuals")
abline(h = 0, col = "red")
title("Figure 1: Residuals vs Fitted")

# 2. QQ plot (for normality)
qqnorm(residuals, main = "Figure 2: Normal Q-Q Plot")
qqline(residuals, col = "red")

Examining the plots:

- **Figure 1: Residuals vs Fitted Plot:** The plot shows a different number of points above and below the reference line and has somewhat of a “cone” shape, suggesting a violation of the linearity and homoscedasticity assumptions. 

- **Figure 2: Q-Q Plot:** The points near the ends of the reference line deviate from the line significantly, suggesting a violation of normality.

Log-transforming the response `realSum` should validate these assumptions. Below, we check for multicollinearity.

In [ ]:
# Main Developer: Alizah

# Find the aliases in the linear model 
# vif() will result in an error if there is perfect multicollinearity anywhere so we have to remove that first
alias_structure <- alias(lm(realSum ~ ., data = data_filt))
alias_vars <- rownames(alias_structure$Complete)
cat("Variables to Remove: ", alias_vars)

We must remove 'room_shared' and 'room_private' in order to run `vif`, as they have perfect multicollinearity.

In [ ]:
# Main Developer: Alizah

data_noalias <- dplyr::select(data_filt, -room_shared, -room_private)
full_model_noalias <- lm(realSum ~ ., data = data_noalias)

cat("Table 1: vif output")
vif(full_model_noalias)

The values under the last column are all $< \sqrt{5} \approx 2.24$, verifying minimal multicollinearity between the remaining predictors.

### Testing & Evaluation:

In [ ]:
# Main developer: Marcus

# Standardize the data by log-transforming realSum and scaling numerical variables
training_data_std <- training_data %>%
    mutate(realSum = log(realSum)) %>%  # Log-transform realSum
    mutate(across(
      .cols = where(is.numeric) & !any_of("realSum"),  # All numeric vars except realSum
      .fns = ~ scale(.)
    )) %>%
    dplyr::select(-room_shared, -room_private) # Drop collinear variables

# We do not peek at the testing data or leak any into our model
testing_data_std <- testing_data %>%
    mutate(realSum = log(realSum)) %>%  # Log-transform realSum
    mutate(across(
      .cols = where(is.numeric) & !any_of("realSum"),  # All numeric vars except realSum
      .fns = ~ scale(.)
    )) %>%
    dplyr::select(-room_shared, -room_private) # Drop collinear variables

In [ ]:
# Main developer: Alizah

# Use the Intercept-Only Model, Standardized, as a Baseline for performance
baseline_model_std <- lm(realSum ~ 1, data = training_data_std)

# Calculate RMSE using baseline model on testing data
predictions <- predict(baseline_model_std, testing_data_std)

rmse_baseline <- rmse(testing_data_std$realSum, predictions)
cat("RMSE Baseline (Scaled): ", rmse_baseline)

actuals_original <- exp(testing_data_std$realSum)
predictions_original <- exp(predictions)
rmse_baseline_original <- rmse(actuals_original, predictions_original)
cat("\n")
cat("RMSE Baseline (Original, in Euros): ", rmse_baseline_original)

In [ ]:
# Main Developer: Alizah

# Run forward selection starting from baseline model (full_model is the upper boundary)
full_model_std <- lm(realSum ~ ., data = training_data_std)

step_model_std <- stepAIC(
    baseline_model_std, 
    scope = list(lower = baseline_model_std, upper = full_model_std),
    direction = "forward",
    trace = FALSE) # minimizes output

# THIS CELL MAY TAKE A WHILE TO RUN

In [ ]:
# Viewing the selected predictors
final_formula_std = formula(step_model_std)
final_formula_std

In [ ]:
# Main Developer: Alizah
# Contributor: Zahra(Style Guide)

# Create Final Prediction Model Using Selected Predictors
final_model = lm(final_formula_std, data = training_data_std)
summary(final_model) # Generate important summary statistics

Above, the least significant predictors are `metro_dist` and `guest_satisfaction_overall`, which remained in the model as `stepAIC` uses AIC rather than p-values to balance fit and complexity. For our inferential purposes, these will be ignored.

### Validating Assumptions

In [ ]:
# Main developer: Alizah
# Collaborator: Zahra (Histogram of Residuals)

# Get Residuals
residuals <- resid(step_model_std)
fitted_vals <- fitted(step_model_std)

# 1. Residuals vs Fitted plot (for linearity & homoscedasticity)
plot(fitted_vals, residuals, xlab = "Fitted values", ylab = "Residuals")
abline(h = 0, col = "red")
title("Figure 3: Residuals vs Fitted for Standardized data")

# 2. QQ plot (for normality)
qqnorm(residuals, main = "Figure 4: Normal Q-Q Plot for Standardized data")
qqline(residuals, col = "red")

- **Figure 3: Residuals vs Fitted Plot, Standardized:** The points above and below the line appear equally distributed in an elliptical distribution, validating the assumption of linearity and homoscedasticity (constant variance). 
- **Figure 4: Q-Q Plot, Standardized:** The points fit better with the reference line, validating the assumption of normality. However, it remains slightly curved upwards at both ends.

Now, let’s evaluate the model.

In [ ]:
# Main developer: Alizah

# Predicting listing prices (realSum) with our model 
predictions <- predict(final_model, testing_data_std)
actuals <- testing_data_std$realSum

# Calculating RMSE
rmse_model <- rmse(testing_data_std$realSum, predictions)

actuals_original <- exp(actuals)
predictions_original <- exp(predictions)
rmse_model_original <- rmse(actuals_original, predictions_original)

In [ ]:
# Main Developer: Alizah
# Collaborator: Zahra

# Compile metrics into a table
metrics <- data.frame(
    metric = c("Baseline RMSE", "Model RMSE"),
    log_value = c(rmse_baseline, rmse_model),
    value = c(rmse_baseline_original, rmse_model_original)
)

cat("Table 1: Result Metrics")
metrics

We can see that the model we created using stepAIC() has a much better RMSE value than our baseline model, which indicates that using our selected variables lead to a more accurate prediction of realSum values as compared to simply using the mean response value. 

As an added visual for our model performance, we plot model predictions against the observed values from our testing set.

In [ ]:
# Main Developer: Zahra 

# Plotting Model Predictions vs. Actuals
# Create plot data
plot_data <- data.frame(actuals = testing_data_std$realSum,
                        predictions = predict(final_model, testing_data_std))

# Extract the baseline model intercept
baseline_intercept <- coef(baseline_model_std)[1]

# Build plot
ggplot(plot_data, aes(x = actuals, y = predictions)) +
  geom_point(aes(color = "Predicted Values")) +
  
  # Ideal y = x line
  geom_abline(aes(color = "Ideal Fit"), slope = 1, intercept = 0, linetype = "dashed", linewidth = 1) +
  
  # Fitted trend line
  geom_smooth(aes(color = "Model Trend"), method = "lm", se = FALSE, linewidth = 1) +
  
  # Horizontal line for baseline model intercept
  geom_hline(aes(yintercept = baseline_intercept, color = "Baseline Model"),
             linetype = "dotdash", linewidth = 1) +
  
  # Manual color and legend mapping
  scale_color_manual(
    name = "Legend",
    values = c(
      "Predicted Values" = "lightblue",
      "Ideal Fit" = "red",
      "Model Trend" = "blue",
      "Baseline Model" = "darkgreen"
    )
  ) +
  
  labs(
    title = "Figure 7: \nActual vs Predicted Prices with Model Trends",
    x = "Actual Log Price",
    y = "Predicted Log Price"
  ) +
  theme_minimal() +
  theme(legend.position = "bottom")


Ideally, if our model has complete accuracy, its trendline would be y=x, which is shown as a dashed diagonal line. In Figure 5, We can see that our model trend line isn’t similar. For actual values lower than mean(realSum), our predictions are a bit higher. For actual values higher than mean(realSum), our predictions are a bit lower.  

# 3) Discussion

Through stepwise variable selection using `stepAIC`, we obtained the formula above. This formula contains 11 of the 15 analysis variables we started with.

We expected to find location, listing size, quality and host reputation to be important factors. From above, we confirm location, specifically distance to the city center (`city`) to be negatively associated with price. We also confirm listing size (`person_capacity`, `bedrooms`, `multi`) and quality (`cleanliness_rating`) to be positively associated with price. However, it appears that host reputation (`host_is_superhost`) was not included in the model as a strong predictor. Moreover, we note the following:

- Rome and Budapest are negatively associated with price. This matches our EDA.
- Private and Shared rooms are negatively associated with price. We would expect Private rooms to be positively associated, but they are not.
- Listings for business use are positively associated with price.
- Listings for weekends are positively associated with price.

Our model had an RMSE $\approx$ `0.29` ($\approx$ 141 $\texteuro$), compared to the baseline with RMSE $\approx$ `0.48` ($\approx$ 83 $\texteuro$). 

## Improvements and Considerations

Below are some considerations for our analysis.

- Assumptions: If some assumptions are proven false, our results may be invalid or require further correction. 
- Overlooked Predictors & Relationships: There may be confounders, hidden interactions or other overlooked predictors excluded from the dataset or our model. For example, important variables could be the season, amenities, or neighbourhood crime rate. We also only briefly touched upon the moderately correlated numerical inputs.
- Part vs. Whole: Only looking at the data for specific cities (Rome, Budapest and London) could result in *Simpson’s paradox*, where the apparent associations for these three cities are not observed in other cities, or in Europe, or the world.

## Future Research

Future studies could examine the long-term trends in Airbnb price or consider more variables and interactions such as the season. However, it is important to note the risk of overfitting when including more predictors in a model. It may also be of interest to analyze Airbnb prices on a per-city basis, as well as the trends for different groupings of cities (ie. Capital cities), or the trends for other continents and countries. 

# 4) References

- Toader, Victor, et al. "Analysis of Price Determinants in the Case of Airbnb Listings." Economic Research-Ekonomska Istraživanja, vol. 35, no. 1, 2021, pp. 2493–2509. https://doi.org/10.1080/1331677x.2021.1962380.

- Gyódi, Karol, and Łukasz Nawaro. "Determinants of Airbnb Prices in European Cities: A Spatial Econometrics Approach." Tourism Management, vol. 86, 2021, https://doi.org/10.1016/j.tourman.2021.104319.

- Gyódi, Karol, and Łukasz Nawaro. "Airbnb Prices in European Cities." Kaggle, 10 Mar. 2024, www.kaggle.com/datasets/thedevastator/airbnb-prices-in-european-cities/data.

